#**LIBRAIRIES**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn import svm #Linear SVM
from sklearn.tree import DecisionTreeClassifier #decision tree
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.linear_model import LogisticRegression #logistic regression

from sklearn.metrics import f1_score #f1score scorer

import random

from sklearn.model_selection import GridSearchCV #grid search

import pickle #saving model

#**Defining the class**

In [2]:
class Review:
  def __init__(self, text, score):
    self.text = text        #review text
    self.score = score      #rate
    self.sentiment = self.get_sentiment()

  #define the sentiment function

  def get_sentiment(self):
    if self.score <= 2:
      return "NEGATIVE"
    elif self.score >= 3:
      return "POSITIVE"
#improving the model 

class ReviewContainer:
  def __init__(self, reviews):
    self.reviews = reviews
  
  def get_text(self):
    return [x.text for x in self.reviews]
  
  def get_sentiment(self):
    return [x.sentiment for x in self.reviews]

  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment == "NEGATIVE", self.reviews))
    positive = list(filter(lambda x: x.sentiment == "POSITIVE", self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk
    random.shuffle(self.reviews)

#**LOAD AND CLEAN THE DATA**

##Cleaning data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/review_data.csv')
for i in range(len(df['Evaluation'])):
  df['Evaluation'][i] = int(df['Evaluation'][i].replace(',0', " "))
df.head()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Nom,Titre,Texte,Evaluation
0,0,Charline V,Parfait,Une huile de super qualité ! Je m'en sert pour...,5
1,1,JO & JOE,DOUBLE USAGE BIO,-----CE QUE CONTIENT LA BOITE------ Une boîte ...,4
2,2,LOULOU,la meilleure huile de coco,J'en ai testé des huiles de coco ! Mais celle ...,5
3,3,Kaizukim,huile que j'utilise pour la peau à eczéma de m...,Your browser does not support HTML5 video.\n\n...,5
4,4,Fernbach,Super,"Commandé le lundi et reçu le mercredi, dans un...",4


In [4]:
print(len(df))
df = df.drop_duplicates(subset=['Texte'], keep ='last')
df = df.dropna(subset=['Texte','Evaluation'])
print(len(df))
df.head()

3938
3688


,Unnamed: 0,Nom,Titre,Texte,Evaluation
0,0,Charline V,Parfait,Une huile de super qualité ! Je m'en sert pour...,5
1,1,JO & JOE,DOUBLE USAGE BIO,-----CE QUE CONTIENT LA BOITE------ Une boîte ...,4
2,2,LOULOU,la meilleure huile de coco,J'en ai testé des huiles de coco ! Mais celle ...,5
3,3,Kaizukim,huile que j'utilise pour la peau à eczéma de m...,Your browser does not support HTML5 video.\n\n...,5
4,4,Fernbach,Super,"Commandé le lundi et reçu le mercredi, dans un...",4


In [5]:
df['Evaluation'][3687]

5

In [32]:
vals = df.values
review = vals.tolist()
print(review)

[[0, 'Charline V', 'Parfait', "Une huile de super qualité ! Je m'en sert pour tout, le corps, les cheveux, le visage en soin et en démaquillant, je m'en sert sur mon bébé, pour mettre dans le bain et même en cuisine. Cette huile a une odeur à tomber par terre et un goût suptile qui agrémente tout les plats. Je vous la recommande si vous aimez la noix de coco sinon il existe des huiles de coco sans odeurs ni goût, parfait pour avoir tous les biens faits sans vous écœurée.Hésitez pas à dire si ce commentaire vous a été utile.", 5], [1, 'JO & JOE', 'DOUBLE USAGE BIO', '-----CE QUE CONTIENT LA BOITE------ Une boîte de 1/2 kg huile de coco (sous forme solide à température ambiante)-----LES PLUS------ Produit bio- Peut être aussi bien utilisé en cuisine qu\'en cosmétique-----BILAN------ Très satisfaite de cet achat car bien moins cher que ce que j\'avais trouvée dans le commerce jusqu\'à ce jour en plus bio et double usage. Je l\'ai seulement utilisé sur mes cheveux, je recommande de le pass

In [33]:
print(review[3][3])
for i in range(len(review)):
  review[3][i] = review[3][i].replace('Your browser does not support HTML5 video.\n\n\n', ' ')
print(review[3][3])

    j'ai commandé cette huile pour la peau atopique de mon fils ... je la trouve agréable a appliquer et relativement efficace .... Ses soucis n'ont pas cessé du jour au lendemain mais au moins cela a permis d'hydrater sa peau avec quelque chose de naturel . Le pot est d'une contenance tout à fait correcte et vous en aurez pour un moment . Attention cependant à la garder dans un endroit protégé de la chaleur car sinon on se retrouve avec un peau d'huile (qui reprend sa texture d'origine quand la température de la pièce redescend)


AttributeError: ignored

In [35]:
review[3][3]

"   \xa0j'ai commandé cette huile pour la peau atopique de mon fils ... je la trouve agréable a appliquer et relativement efficace .... Ses soucis n'ont pas cessé du jour au lendemain mais au moins cela a permis d'hydrater sa peau avec quelque chose de naturel . Le pot est d'une contenance tout à fait correcte et vous en aurez pour un moment . Attention cependant à la garder dans un endroit protégé de la chaleur car sinon on se retrouve avec un peau d'huile (qui reprend sa texture d'origine quand la température de la pièce redescend)"

In [38]:
for i in range(len(review)):
    reviews.append(Review(review[i][3],review[i][4]))

In [44]:
print(reviews[2].score)
print(reviews[2].sentiment)
print(len(reviews))

5
POSITIVE
3723


#**PREPARING THE DATA**

In [92]:
#split the data train and test
training, test = train_test_split(reviews, test_size = 0.30, random_state = 42 )

#improving the model by making number of positive reviews = negative reviews#
training_container = ReviewContainer(training)
testing_container =  ReviewContainer(test)

training_container.evenly_distribute()
testing_container.evenly_distribute()


train_x = training_container.get_text()
train_y = training_container.get_sentiment()

test_x = testing_container.get_text()
test_y = testing_container.get_sentiment()

print(train_y.count("POSITIVE"))
print(train_y.count("NEGATIVE"))
print(test_y.count("POSITIVE"))
print(test_y.count("NEGATIVE"))

207
207
78
78


In [82]:
train_y.count('NEGATIVE')

207

#**USING BAG OF WORDS VECTORIZATION FOR THE TEXT**

In [93]:
vectorizer = TfidfVectorizer()
tarin_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

print(train_x[22])
print(tarin_x_vectors[22].toarray())

vendeur serieux tres bon produit
[[0. 0. 0. ... 0. 0. 0.]]


#**CLASSIFICATION**

##USING LINEAR SVM

In [97]:
clf_svm = svm.SVC(kernel = 'linear' )
clf_svm.fit(tarin_x_vectors, train_y)
#prediction just testing
print(test_y[10])
print(clf_svm.predict(test_x_vectors[10]))

POSITIVE
['NEGATIVE']


##USING DECISION TREE

In [95]:
clf_dec_tree = DecisionTreeClassifier() 
clf_dec_tree.fit(tarin_x_vectors, train_y)
#prediction just testing
print(test_y[10])
print(clf_dec_tree.predict(test_x_vectors[10]))

POSITIVE
['NEGATIVE']


##USING LOGISTIC REGRESSION

In [98]:
clf_log_reg = LogisticRegression() 
clf_log_reg.fit(tarin_x_vectors, train_y)
#prediction just testing
print(test_y[10])
print(clf_log_reg.predict(test_x_vectors[10]))

POSITIVE
['NEGATIVE']


##USING NAIVE BAYES

In [99]:
clf_nb= GaussianNB() 
clf_nb.fit(tarin_x_vectors.toarray(), train_y)
#prediction just testing
print(test_y[10])
print(clf_nb.predict(test_x_vectors[10].toarray()))

POSITIVE
['POSITIVE']


#**ANALYSIS AND EVALUATION**

##Evaluation using Mean Accuracy

In [100]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec_tree.score(test_x_vectors, test_y))
print(clf_log_reg.score(test_x_vectors, test_y))
print(clf_nb.score(test_x_vectors.toarray(), test_y))

0.8269230769230769
0.7435897435897436
0.8461538461538461
0.7307692307692307


##F1 Scores

In [101]:
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average = None, labels = ['NEGATIVE','POSITIVE']))#2
print(f1_score(test_y, clf_dec_tree.predict(test_x_vectors), average = None, labels = ['NEGATIVE','POSITIVE']))#3
print(f1_score(test_y, clf_log_reg.predict(test_x_vectors), average = None, labels = ['NEGATIVE','POSITIVE'])) #1
print(f1_score(test_y, clf_nb.predict(test_x_vectors.toarray()), average = None, labels = ['NEGATIVE','POSITIVE']))#4
#AS we can see our model is good for predicting the postive reviews but bad for the negative reviews

[0.84210526 0.80851064]
[0.7260274  0.75903614]
[0.85714286 0.83333333]
[0.72       0.74074074]


##IMPROVING THE MODEL

In [85]:
a = test_y.count("POSITIVE")
b = test_y.count("NEGATIVE")

c = train_y.count("NEGATIVE")
d = train_y.count("POSITIVE")

print(a+d)
print(b+c)

#WE NEED 285 COMMENT EACH (NEGATIVE AND POSITIVE)
#I think we need more data , the problem is with the negative class we had more positive commentds than negative and that's why we saw with the f1_score method the our model know how to predict the positive class more than the negative
#the solution is to make our data 50% positive and 50 % negative, and it's also a problem because of the small amount of negative commentes

3438
285


#**TESTING OUR MODEL**

In [106]:
test_set = ["j'aime pas ce produit", "N'achetez pas ce produit","c'est bon", "je le recommande", "je le recommande pas"]
new_test = vectorizer.transform(test_set)

clf_log_reg.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE'],
      dtype='<U8')